In [1]:
import hashlib
import io
import logging
import os
from os import walk
import pickle
import PIL.Image
import time

from object_detection.utils import dataset_util

import tensorflow as tf
from tensorflow import keras
print('Tensorflow version: ', tf.__version__)
print("Eager execution: ", tf.executing_eagerly())
print("Listing CPUs: ", tf.config.list_physical_devices('CPU'))
print("Listing GPUs: ", tf.config.list_physical_devices('GPU'))
print("Listing XLA_GPUs: ", tf.config.list_physical_devices('XLA_GPU'))

DATA = "./raw_dataset"
PNG_FOLDER = "png_img"
LABELS_FOLDER = "labels"
IDS_FOLDER = "./ids"
OUTPUT_PATH = "./dataset/dataset.record"
IMG_TYPE = "png"

Tensorflow version:  2.1.0
Eager execution:  True
Listing CPUs:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Listing GPUs:  []
Listing XLA_GPUs:  [PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU')]


In [2]:
# LISTING ALL EXAMPLES FROM THE IMAGE FOLDER
img_path = os.path.join(DATA, PNG_FOLDER)
images = []
for (dirpath, dirnames, filenames) in walk(img_path):
    for filename in filenames:
        splitted = filename.split('.')
        if len(splitted) and splitted[1].lower()=="png":
            images.append([splitted[0], splitted[1]])

In [3]:
def create_tf_example(img_file, annotation, map_id):
    ## LOADING IMAGE
    filename = img_file[0]+"."+img_file[1]
    img_path = os.path.join(DATA, PNG_FOLDER, filename)
    with tf.io.gfile.GFile(img_path, 'rb') as fid:
        encoded_image_data = fid.read()
    # Checking format
    encoded_image_io = io.BytesIO(encoded_image_data)
    image = PIL.Image.open(encoded_image_io)
    if image.format != 'PNG':
        raise ValueError('Image format not PNG')
    # Hashing
    key = hashlib.sha256(encoded_image_data).hexdigest()
    
    ## GENERAL FEATURES
    width, height = image.size
    image_format = IMG_TYPE.encode('utf8') # b'jpeg' or b'png'
    NameError('HiThere')
    ## DEFINING BOUNDING BOXES
    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)
    for bb in annotation:
        xmins.append(bb["xmin"]/width)
        xmaxs.append(bb["xmax"]/width)
        ymins.append(bb["ymin"]/height)
        ymaxs.append(bb["ymax"]/height)
        classes_text.append(bb["taxon"].encode('utf8'))
        classes.append(map_id[bb["taxon"]])
        
        
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [11]:
# CREATE TF RECORD FILE
id_map = pickle.load(open(os.path.join(DATA, IDS_FOLDER, "0000.pickle"), "rb" ))
writer = tf.io.TFRecordWriter(OUTPUT_PATH)
i = 0;
for image in images:
    if (i/len(images))%10==0:
        print(int(i/len(images))*100,"%")
    annotation_path = os.path.join(DATA, LABELS_FOLDER, image[0]+".pickle")
    if os.path.isfile(annotation_path):
        annotation = pickle.load(open(annotation_path, "rb"))
        tf_example = create_tf_example(image, annotation, id_map)
        writer.write(tf_example.SerializeToString())
    else:
        print("WARNING: Image ", name, ".", extension, " has no associated annotation file. Ignoring image.")
    i += 1
writer.close()

0 %


In [ ]:
print(pickle.load(open(os.path.join(DATA, LABELS_FOLDER, "0000.pickle"), "rb")))

In [8]:
len(images)

200